# Data Frame Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [3]:
import pyspark.sql
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, IntegerType
from pyspark.sql.functions import udf, desc, asc, count, avg, col, sort_array, isnan, when
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.window import Window
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
spark = SparkSession.builder.appName('Data Wrangling').getOrCreate()
path = "data/sparkify_log_small.json"
log = spark.read.json(path)

In [2]:
log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



# Question 1

Which page did user id "" (empty string) NOT visit?

In [6]:
all_page = log.select('page').distinct().sort('page').collect()

In [7]:
blank_page = log.filter(log.userId == "").select('page').distinct().sort('page').collect()

In [8]:
for row in (set(all_page) - set(blank_page)):
    print(row.page)

Error
Submit Upgrade
Save Settings
NextSong
Logout
Downgrade
Settings
Submit Downgrade
Upgrade


# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [15]:
log.filter(log.userId == "").count()

336

# Question 3

How many female users do we have in the data set?

In [16]:
log.take(1)

[Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046')]

In [17]:
log.filter(log.gender == 'F').select('userId').distinct().count()

462

# Question 4

How many songs were played from the most played artist?

In [11]:
log.groupby(log.artist).count().sort(desc('count')).show()

+--------------------+-----+
|              artist|count|
+--------------------+-----+
|                null| 1653|
|            Coldplay|   83|
|       Kings Of Leon|   69|
|Florence + The Ma...|   52|
|            BjÃÂ¶rk|   46|
|       Dwight Yoakam|   45|
|       Justin Bieber|   43|
|      The Black Keys|   40|
|         OneRepublic|   37|
|                Muse|   36|
|        Jack Johnson|   36|
|           Radiohead|   31|
|        Taylor Swift|   29|
|Barry Tuckwell/Ac...|   28|
|          Lily Allen|   28|
|               Train|   28|
|           Daft Punk|   27|
|           Metallica|   27|
|          Nickelback|   27|
|          Kanye West|   26|
+--------------------+-----+
only showing top 20 rows



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [5]:
function = udf(lambda x: 1 if x == 'Home' else 0)

In [6]:
windowval = Window \
    .partitionBy("userId") \
    .orderBy('ts') \
    .rangeBetween(Window.unboundedPreceding, 0)


In [7]:
cumsum = log.filter((log.page == 'NextSong') | (log.page == 'Home')) \
            .select('userId', 'ts', 'page') \
            .withColumn('home_page', function('page')) \
            .withColumn("period", Fsum("home_page").over(windowval))            

In [14]:
cumsum.filter(cumsum.page == 'NextSong') \
        .groupBy('userId', 'period') \
        .agg({'period' : 'count'}) \
        .agg({'count(period)' : 'avg'}) \
        .show()


+------------------+
|avg(count(period))|
+------------------+
|6.9558333333333335|
+------------------+

